# METODE ZA PODEŠAVANJE HIPERPARAMETARA

#Grid Search

In [4]:
!pip install scikeras
!pip install scikit-learn==1.4.2
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.5 MB/s eta 0:00:00


In [12]:
from tensorflow import keras
from tensorflow.keras import layers
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
from skopt import BayesSearchCV
from sklearn.model_selection import PredefinedSplit
import pandas as pd
import plotly.graph_objects as go

In [6]:
# Učitavanje Fashion-MNIST dataseta
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

print('Broj uzoraka u skupu za treniranje i testiranje:')
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print('\n')

class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

print('Klase u datasetu:')
for i in range (len(class_names)):
    print(f"{i}: {class_names[i]}")

# Normalizacija (0–255 → 0–1)
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Pretvaranje slika 28x28 u vektore duljine 784
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Broj uzoraka u skupu za treniranje i testiranje:
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


Klase u datasetu:
0: T-shirt/top
1: Trouser
2: Pullover
3: Dress
4: Coat
5: Sandal
6: Shirt
7: Sneaker
8: Bag
9: Ankle boot


In [7]:
def create_model(neurons=128, learning_rate=0.001):
    model = keras.Sequential()

    model.add(layers.Dense(neurons, activation="relu", input_shape=(784,)))
    model.add(layers.Dense(neurons // 2, activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))

    opt = keras.optimizers.Adam(learning_rate=learning_rate) # opisati zašto je Adam optimizer korišten

    model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy", #Opisati zašto je ovo korišteno
        metrics=["accuracy"]
    )

    return model

#MANUAL SEARCH

In [14]:
model = KerasClassifier(model=create_model, epochs=10, verbose=0)

manual_configs = [
    {"neurons": 32,  "learning_rate": 0.01,   "batch_size": 32},
    {"neurons": 64,  "learning_rate": 0.001,  "batch_size": 64},
    {"neurons": 128, "learning_rate": 0.001,  "batch_size": 128},
    {"neurons": 256, "learning_rate": 0.0001, "batch_size": 256},

    {"neurons": 32,  "learning_rate": 0.0001,  "batch_size": 128},
    {"neurons": 64,  "learning_rate": 0.01, "batch_size": 32},
    {"neurons": 128, "learning_rate": 0.001, "batch_size": 256},
]

manual_results = []

for config in manual_configs:
    print(f"Testiram model: ", config)

    model.set_params(
        model__neurons=config['neurons'],
        model__learning_rate=config['learning_rate']
    )

    # Treniranje
    history = model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=config['batch_size'],
        verbose=0,
        validation_split=0.2
    )

    # Evaluacija na testnom skupu
    score = model.score(X_test, y_test)
    manual_results.append(score)
    print(f"   Točnost: {score:.4f}")

# Pronalazak najboljeg manualnog rezultata
best_manual_idx = np.argmax(manual_results)
best_manual_score = manual_results[best_manual_idx]

Testiram model:  {'neurons': 32, 'learning_rate': 0.01, 'batch_size': 32}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



   Točnost: 0.8456
Testiram model:  {'neurons': 64, 'learning_rate': 0.001, 'batch_size': 64}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



   Točnost: 0.8753
Testiram model:  {'neurons': 128, 'learning_rate': 0.001, 'batch_size': 128}


...

   Točnost: 0.8322
Testiram model:  {'neurons': 64, 'learning_rate': 0.01, 'batch_size': 32}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



   Točnost: 0.8516
Testiram model:  {'neurons': 128, 'learning_rate': 0.001, 'batch_size': 256}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



   Točnost: 0.8772


PRIKAZ MS

In [15]:
neurons_m = [c['neurons'] for c in manual_configs]
lr_m = [c['learning_rate'] for c in manual_configs]
bs_m = [c['batch_size'] for c in manual_configs]
labels_m = [f"N:{c['neurons']}, LR:{c['learning_rate']}, B:{c['batch_size']}" for c in manual_configs]

fig_3d_m = go.Figure()

fig_3d_m.add_trace(go.Scatter3d(
    x=neurons_m,
    y=lr_m,
    z=manual_results,
    mode='markers+text',
    marker=dict(size=10, color=manual_results, colorscale='Viridis', showscale=True),
    text=labels_m,
    name='Manualne točke'
))

fig_3d_m.update_layout(
    title='Manual Search - Pozicije u prostoru parametara',
    scene=dict(xaxis_title='Neurons', yaxis_title='Learning Rate', zaxis_title='Accuracy')
)
fig_3d_m.show()

fig_bar_m = go.Figure(data=[
    go.Bar(
        x=labels_m,
        y=manual_results,
        text=[f"{s:.4f}" for s in manual_results],
        textposition='auto',
        marker_color='indianred'
    )
])

fig_bar_m.update_layout(
    title='Manual Search: Usporedba odabranih konfiguracija',
    xaxis_title='Konfiguracija',
    yaxis_title='Accuracy',
    yaxis=dict(range=[min(manual_results)-0.02, max(manual_results)+0.02]) # Fokus na razlike
)
fig_bar_m.show()

#GRID SEARCH

In [16]:
model = KerasClassifier(model=create_model, epochs=10, verbose=0)

param_grid = {
    "model__neurons": [32, 64, 128],
    "model__learning_rate": [0.0001, 0.001, 0.01],
    "batch_size": [32, 64]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring="accuracy", n_jobs=-1)

grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



KeyboardInterrupt: 

PRIKAZ GS

In [17]:
# 3D prikaz rezultata grid searcha
params = grid_result.cv_results_['params']
scores = grid_result.cv_results_['mean_test_score']

neurons = np.array([p['model__neurons'] for p in params])
learning_rate = np.array([p['model__learning_rate'] for p in params])
accuracy = np.array(scores)
batch_size = np.array([p['batch_size'] for p in params])

fig = go.Figure()

# --- Batch size = 32 ---
mask_32 = batch_size == 32
fig.add_trace(go.Scatter3d(
    x=neurons[mask_32],
    y=learning_rate[mask_32],
    z=accuracy[mask_32],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Batch size = 32'
))

# --- Batch size = 64 ---
mask_64 = batch_size == 64
fig.add_trace(go.Scatter3d(
    x=neurons[mask_64],
    y=learning_rate[mask_64],
    z=accuracy[mask_64],
    mode='markers',
    marker=dict(size=6, color='green'),
    name='Batch size = 64'
))

# --- Najbolja kombinacija ---
best_idx = np.argmax(accuracy)
fig.add_trace(go.Scatter3d(
    x=[neurons[best_idx]],
    y=[learning_rate[best_idx]],
    z=[accuracy[best_idx]],
    mode='text',
    text=['Optimum'],
    textposition='top center',
    name='Optimum'
))

fig.update_layout(
    title='Grid Search - Adam Optimizer',
    scene=dict(
        xaxis_title='Neurons',
        yaxis_title='Learning rate',
        zaxis_title='Accuracy'
    )
)

fig.show()

NameError: name 'grid_result' is not defined

In [ ]:
print("Najbolja točnost - GS:")
print(grid_result.best_score_)

print("\nNajbolji hiperparametri - GS:")
print(grid_result.best_params_)

best_model_gs = grid_result.best_estimator_
test_accuracy_gs = best_model_gs.score(X_test, y_test)

print("\nTočnost na test skupu - GS:")
print(test_accuracy_gs)

Najbolja točnost - GS:
0.8864333333333333

Najbolji hiperparametri - GS:
{'batch_size': 64, 'model__learning_rate': 0.001, 'model__neurons': 128}

Točnost na test skupu - GS:
0.8765


#RANDOMIZED SEARCH

In [ ]:
model = KerasClassifier(model=create_model, epochs=10, verbose=0)

param_dist = {
    "model__neurons": [32, 64, 128],
    "model__learning_rate": loguniform(1e-4, 1e-2),  # kontinuirani raspon
    "batch_size": [32, 64]
}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=20,        # broj nasumičnih kombinacija koje želimo isprobati
    cv=3,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42
)

random_result = random_search.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


PRIKAZ RS

In [ ]:
# Parametri i rezultati
params_rs = random_result.cv_results_['params']
scores_rs = random_result.cv_results_['mean_test_score']

neurons_rs = np.array([p['model__neurons'] for p in params_rs])
learning_rate_rs = np.array([p['model__learning_rate'] for p in params_rs])
accuracy_rs = np.array(scores_rs)
batch_size_rs = np.array([p['batch_size'] for p in params_rs])

fig_rs = go.Figure()

# Batch size = 32
mask_32_rs = batch_size_rs == 32
fig_rs.add_trace(go.Scatter3d(
    x=neurons_rs[mask_32_rs],
    y=learning_rate_rs[mask_32_rs],
    z=accuracy_rs[mask_32_rs],
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Batch size = 32'
))

# Batch size = 64
mask_64_rs = batch_size_rs == 64
fig_rs.add_trace(go.Scatter3d(
    x=neurons_rs[mask_64_rs],
    y=learning_rate_rs[mask_64_rs],
    z=accuracy_rs[mask_64_rs],
    mode='markers',
    marker=dict(size=6, color='green'),
    name='Batch size = 64'
))

# Najbolja kombinacija
best_idx_rs = np.argmax(accuracy_rs)
fig_rs.add_trace(go.Scatter3d(
    x=[neurons_rs[best_idx_rs]],
    y=[learning_rate_rs[best_idx_rs]],
    z=[accuracy_rs[best_idx_rs]],
    mode='text',
    text=['Optimum'],
    textposition='top center',
    name='Optimum'
))

fig_rs.update_layout(
    title='Random Search - Adam Optimizer',
    scene=dict(
        xaxis_title='Neurons',
        yaxis_title='Learning rate',
        zaxis_title='Accuracy'
    )
)

fig_rs.show()

In [ ]:
print("Najbolja točnost - RS:")
print(random_result.best_score_)

print("\nNajbolji hiperparametri - RS:")
print(random_result.best_params_)

best_model_rs = random_result.best_estimator_
test_accuracy_rs = best_model_rs.score(X_test, y_test)

print("\nTočnost na test skupu - RS:")
print(test_accuracy_rs)

Najbolja točnost (CV) - Random Search:
0.8863833333333334

Najbolji hiperparametri - Random Search:
{'batch_size': 32, 'model__learning_rate': np.float64(0.0002895927274708841), 'model__neurons': 128}

Točnost na test skupu - Random Search:
0.8764


#BAYESIAN OPTIMIZATION

In [ ]:
print("\n--- Pokretanje: Bayesian Optimization ---")
# Definiramo prostor pretraživanja (ovdje koristimo categorical i real klase)
from skopt.space import Real, Categorical, Integer

search_spaces = {
    'model__neurons': Integer(32, 256),
    'model__learning_rate': Real(1e-4, 1e-2, prior='log-uniform'),
    'batch_size': Categorical([32, 64, 128])
}

bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=15, # Broj iteracija (manje nego kod Random searcha jer je "pametniji")
    cv=3,
    n_jobs=-1,
    random_state=42
)

bayes_result = bayes_search.fit(X_train, y_train)

print(f"Najbolja točnost (Bayesian): {bayes_result.best_score_}")
print(f"Najbolji parametri (Bayesian): {bayes_result.best_params_}")

#PRIKAZ BO

In [ ]:
params_bo = bayes_result.cv_results_['params']
scores_bo = bayes_result.cv_results_['mean_test_score']

neurons_bo = np.array([p['model__neurons'] for p in params_bo])
learning_rate_bo = np.array([p['model__learning_rate'] for p in params_bo])
accuracy_bo = np.array(scores_bo)

fig_bo = go.Figure()

fig_bo.add_trace(go.Scatter3d(
    x=neurons_bo,
    y=learning_rate_bo,
    z=accuracy_bo,
    mode='markers',
    marker=dict(
        size=7,
        color=accuracy_bo,                # Boja ovisi o točnosti
        colorscale='Viridis',
        opacity=0.8,
        colorbar=dict(title="Accuracy")
    ),
    name='Bayesian Trials'
))

# Isticanje najboljeg rezultata
best_idx_bo = np.argmax(accuracy_bo)
fig_bo.add_trace(go.Scatter3d(
    x=[neurons_bo[best_idx_bo]],
    y=[learning_rate_bo[best_idx_bo]],
    z=[accuracy_bo[best_idx_bo]],
    mode='markers',
    marker=dict(size=12, color='red', symbol='diamond'),
    name='Best Bayesian'
))

fig_bo.update_layout(
    title='Bayesian Optimization Results',
    scene=dict(
        xaxis_title='Neurons',
        yaxis_title='Learning Rate',
        zaxis_title='Accuracy'
    )
)

fig_bo.show()